In [1]:
import numpy
import binascii
import cv2, base64
import numpy as np
from pprint import pprint

In [2]:
def encode(text, key):
    enc = []
    for i in range(len(text)):
        key_c = key[i % len(key)]
        enc_c = chr((ord(text[i]) + ord(key_c)) % 256)
        enc.append(enc_c)
    enc = base64.urlsafe_b64encode("".join(enc).encode()).decode()
    return enc

def decode(enc, key):
    dec = []
    enc = base64.urlsafe_b64decode(enc).decode()
    for i in range(len(enc)):
        key_c = key[i % len(key)]
        dec_c = chr((256 + ord(enc[i]) - ord(key_c)) % 256)
        dec.append(dec_c)
    dec = "".join(dec)
    return dec

def text2bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    Text2bits = bits.zfill(8 * ((len(bits) + 7) // 8))
    return Text2bits

def bits2text(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    Bits2text = n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'
    return Bits2text

def chunks(string, num):
    """Produce `n`-character chunks from `s`."""
    for start in range(0, len(string), num):
        yield string[start: start+num]

In [3]:
class Text:
    def __init__(self, text, n=2):
        self.text = text
        self.n = n
    
    encrypt = lambda self, key: encode(self.text, key)
    decrypt = lambda self, key: decode(self.text, key)
    text2bits = lambda self, text: text2bits(text)
    bits2text = lambda self, bits: bits2text(bits)
    int2bin = lambda self, x: format(x, 'b')
    bin2int = lambda self, x: int(format(int(x, 2), 'd'))
    
    def genBinCode(self, key):
        encodedText = self.encrypt(key)
        encodedBinaryText = self.text2bits(encodedText)
        encodedLengthGroup = int(len(encodedBinaryText)/int(self.n))
        encodedBitGroup = self.int2bin(encodedLengthGroup).zfill(24)
        self.insertString = encodedBitGroup+ encodedBinaryText
        return encodedBitGroup, encodedBinaryText
        
    def elementData(self, value, data):
        binary = self.int2bin(value)[:-int(self.n)] + data
        return self.bin2int(binary)
    
    def readImage(self, filename):
        self.filename = filename
        self.image = cv2.imread(filename)
    
    def changeImage(self):
        insertStringList, image = [], self.image
        insertString = self.insertString
        for chunk in chunks(insertString, int(self.n)): insertStringList.append(chunk)
            
        length, count = len(insertStringList), 0
        for i in range(len(image)):
            for j in range(len(image[i])):
                for k in range(3):
                    if count< length: 
                        data = insertStringList[count]
                        count += 1
                        element = image[i][j][k]
                        element = self.elementData(element, data)
                        image[i][j][k] = element
                    else:
                        image[i][j][k] = image[i][j][k]        
        self.image = image
    
    def saveImage(self):
        cv2.imwrite('changed-'+self.filename, self.image)

In [9]:
texts = '''Text messaging, or texting, is the act of composing and sending electronic messages, typically consisting of alphabetic and numeric characters, between two or more users of mobile phones, tablets, desktops/laptops, or other devices. Text messages may be sent over a cellular network, or may also be sent via an Internet connection.

The term originally referred to messages sent using the Short Message Service (SMS). It has grown beyond alphanumeric text to include multimedia messages (known as MMS) containing digital images, videos, and sound content, as well as ideograms known as emoji (happy faces, sad faces, and other icons).

As of 2017, text messages are used by youth and adults for personal, family and social purposes and in business. Governmental and non-governmental organizations use text messaging for communication between colleagues. As with emailing, in the 2010s, the sending of short informal messages has become an accepted part of many cultures.[1] This makes texting a quick and easy way to communicate with friends and colleagues, including in contexts where a call would be impolite or inappropriate (e.g., calling very late at night or when one knows the other person is busy with family or work activities). Like e-mail and voice mail, and unlike calls (in which the caller hopes to speak directly with the recipient), texting does not require the caller and recipient to both be free at the same moment; this permits communication even between busy individuals. Text messages can also be used to interact with automated systems, for example, to order products or services from e-commerce websites, or to participate in online contests. Advertisers and service providers use direct text marketing to send messages to mobile users about promotions, payment due dates, and other notifications instead of using postal mail, email, or voicemail.
'''*7

In [10]:
a = Text(texts, n=4)
a.genBinCode('password24')
a.readImage('abc.png')
a.changeImage()
a.saveImage()